# Plotting the results

In [2]:
import pandas as pd
import numpy as np
import os

In [4]:
path = os.path.dirname(os.getcwd())

# Perline (Simple prompt LLMs vs. OCR/HTR)

In [6]:
# Files
scores = {}
for file in os.listdir(path + '/results/scores_comparisons/perline_eval'):
    if file.endswith('.csv'):
        scores[file] = pd.read_csv(path + '/results/scores_comparisons/perline_eval/' + file)


In [8]:
scores['cer_pytesseractOCR_perline.csv']

,Unnamed: 0,id,cer
0,0,1_0,0.804428
1,1,1_1,0.853659
2,2,1_2,1.000000
3,3,1_3,0.762963
4,4,1_4,0.873563
...,...,...,...
278,278,20_9,0.739837
279,279,20_10,0.758621
280,280,20_11,0.948718
281,281,20_12,0.944444
